In [33]:
import cv2
import numpy as np

# Function to add text with red color
def add_text(image, text, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5  # Decrease font size
    font_color = (0, 0, 255)  # Red color
    font_thickness = 1  # Decrease font thickness
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    
    # Calculate the position at the right-bottom of the image
    text_x = image.shape[1] - text_size[0] - 10
    text_y = image.shape[0] - 10
    
    # Add the text to the image
    cv2.putText(image, text, (text_x, position), font, font_scale, font_color, font_thickness, lineType=cv2.LINE_AA)

# Function 1: Smallest Circle that Encapsulates the Particle
def find_encapsulating_circle(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        center = (int(x), int(y))
        radius = int(radius)
        result_image = input_image.copy()
        cv2.circle(result_image, center, radius, (0, 0, 255), 2)
        add_text(result_image, 'Smallest Circle', 20)
        return result_image
    else:
        return input_image

# Function 2: Total Surface Area of the Particle
def calculate_surface_area(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        particle_area = cv2.contourArea(largest_contour)
        result_image = input_image.copy()
        add_text(result_image, 'Total Surface Area', 20)
        add_text(result_image, f'Area: {int(particle_area)} pixels^2', 40)
        return result_image
    else:
        return input_image

def find_major_axis(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        result_image = input_image.copy()

        # Calculate the principal component of the particle
        center, axes, angle = cv2.fitEllipse(largest_contour)
        major_axis_length = max(axes)

        # Calculate the endpoints of the major axis
        angle_rad = np.radians(angle)
        major_axis_endpoint1 = (
            int(center[0] - major_axis_length / 2 * np.cos(angle_rad)),
            int(center[1] - major_axis_length / 2 * np.sin(angle_rad))
        )
        major_axis_endpoint2 = (
            int(center[0] + major_axis_length / 2 * np.cos(angle_rad)),
            int(center[1] + major_axis_length / 2 * np.sin(angle_rad))
        )

        # Draw the major axis on the image with a red color
        cv2.line(result_image, major_axis_endpoint1, major_axis_endpoint2, (0, 0, 255), 2)

        # Add the text label
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_color = (0, 0, 255)  # Red color
        font_thickness = 1
        label = f'Major axis of the particle (in pixels): {int(major_axis_length)}'
        text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
        text_x = input_image.shape[1] - text_size[0] - 10
        text_y = input_image.shape[0] - 10
        cv2.putText(result_image, label, (text_x, text_y), font, font_scale, font_color, font_thickness, lineType=cv2.LINE_AA)

        return result_image
    else:
        return input_image


def calculate_perimeter(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        perimeter = cv2.arcLength(largest_contour, True)
        result_image = input_image.copy()

        # Draw the perimeter on the image with a red color
        cv2.drawContours(result_image, [largest_contour], -1, (0, 0, 255), 2)

        # Add the text label
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_color = (0, 0, 255)  # Red color
        font_thickness = 1
        label = f'Total perimeter of the particle (in pixels): {int(perimeter)}'
        text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
        text_x = input_image.shape[1] - text_size[0] - 10
        text_y = input_image.shape[0] - 10
        cv2.putText(result_image, label, (text_x, text_y), font, font_scale, font_color, font_thickness, lineType=cv2.LINE_AA)

        return result_image
    else:
        return input_image


# Function 5: Centroid of the Particle with Coordinates
def find_centroid(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest_contour)
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
        result_image = input_image.copy()
        
        # Draw the centroid as a red point
        cv2.drawMarker(result_image, (centroid_x, centroid_y), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=1, line_type=cv2.LINE_AA)
        
        # Display the x, y coordinates at the right-bottom of the image
        text = f'Centroid: ({centroid_x}, {centroid_y})'
        add_text(result_image, text, 40)
        return result_image
    else:
        return input_image

def process_and_save_images(input_folder, output_folder):
    # Iterate over the files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpeg") or filename.endswith(".png"):
            input_path = os.path.join(input_folder, filename)
            input_image = cv2.imread(input_path)

            # Apply the five operations to the input image
            encapsulating_circle = find_encapsulating_circle(input_image)
            surface_area = calculate_surface_area(input_image)
            major_axis = find_major_axis(input_image)
            perimeter = calculate_perimeter(input_image)
            centroid = find_centroid(input_image)

            # Create a folder in the output directory for each image
            image_output_folder = os.path.join(output_folder, os.path.splitext(filename)[0])
            os.makedirs(image_output_folder, exist_ok=True)

            # Save the processed images to the respective output folder
            cv2.imwrite(os.path.join(image_output_folder, 'encapsulating_circle.jpg'), encapsulating_circle)
            cv2.imwrite(os.path.join(image_output_folder, 'surface_area.jpg'), surface_area)
            cv2.imwrite(os.path.join(image_output_folder, 'major_axis.jpg'), major_axis)
            cv2.imwrite(os.path.join(image_output_folder, 'perimeter.jpg'), perimeter)
            cv2.imwrite(os.path.join(image_output_folder, 'centroid.jpg'), centroid)


input_folder = 'D:/downloads m/Cognida Coding Challenge/Cognida Coding Challenge/Mineral Processing Technology - Image Analytics/input'  # Input folder containing images
output_folder = 'C:/Users/Roop shankar/OneDrive/Desktop/temp/out'  # Output folder for saving processed images
process_and_save_images(input_folder, output_folder)
